<a href="https://colab.research.google.com/github/AhmedRahim01/car-type-classification/blob/main/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================
# 1️⃣ Imports
# ======================================
import os, shutil
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import TopKCategoricalAccuracy


In [ ]:
import os
import random
import shutil
import scipy.io
import kagglehub

# ===============================
# 1️⃣ تحميل الداتاسيت
# ===============================
print("Downloading dataset...")
DATASET_PATH = kagglehub.dataset_download(
    "eduardo4jesus/stanford-cars-dataset"
)
print("Dataset path:", DATASET_PATH)

# ===============================
# 2️⃣ اكتشاف المسارات تلقائيًا
# ===============================
IMAGES_PATH = None
ANNOS_PATH  = None

for root, dirs, files in os.walk(DATASET_PATH):
    if "cars_train_annos.mat" in files:
        ANNOS_PATH = os.path.join(root, "cars_train_annos.mat")
    if os.path.basename(root) == "cars_train" and "cars_train" in root:
        # folder that contains images
        IMAGES_PATH = root

print("Images path:", IMAGES_PATH)
print("Annotations path:", ANNOS_PATH)

if IMAGES_PATH is None or ANNOS_PATH is None:
    raise FileNotFoundError("❌ Could not locate images or annotations. Dataset structure is unexpected.")

# ===============================
# 3️⃣ إعداد output
# ===============================
OUTPUT_PATH = "/kaggle/working/cars_100_classes"
os.makedirs(OUTPUT_PATH, exist_ok=True)

# ===============================
# 4️⃣ تحميل الـ annotations
# ===============================
annos = scipy.io.loadmat(ANNOS_PATH)
annotations = annos["annotations"][0]

print("Total images:", len(annotations))

# ===============================
# 5️⃣ تجميع الصور حسب class
# ===============================
class_dict = {}

for ann in annotations:
    img_name = ann[5][0]
    class_id = int(ann[4][0][0])

    class_dict.setdefault(class_id, []).append(img_name)

print("Total classes:", len(class_dict))

# ===============================
# 6️⃣ اختيار 100 class
# ===============================
random.seed(42)
selected_classes = random.sample(list(class_dict.keys()), 100)
print("Selected classes:", len(selected_classes))

# ===============================
# 7️⃣ نسخ الصور
# ===============================
missing = 0

for cls in selected_classes:
    class_folder = os.path.join(OUTPUT_PATH, f"class_{cls}")
    os.makedirs(class_folder, exist_ok=True)

    for img in class_dict[cls]:
        src = os.path.join(IMAGES_PATH, img)
        dst = os.path.join(class_folder, img)

        if os.path.exists(src):
            shutil.copy(src, dst)
        else:
            missing += 1

print(f"🔥 DONE! Dataset ready at: {OUTPUT_PATH}")
print(f"⚠️ Missing images (ignored safely): {missing}")


Using Colab cache for faster access to the 'stanford-cars-dataset' dataset.
Dataset path: /kaggle/input/stanford-cars-dataset
Images path: /kaggle/input/stanford-cars-dataset/cars_train/cars_train
Annotations path: /kaggle/input/stanford-cars-dataset/car_devkit/devkit/cars_train_annos.mat
Total images: 8144
Total classes: 196
Selected classes: 100
🔥 DONE! Dataset ready at: /kaggle/working/cars_100_classes
⚠️ Missing images (ignored safely): 0


In [ ]:
# ======================================
# 2️⃣ Create 75 Classes Dataset
# ======================================
SRC_PATH = "/kaggle/working/cars_100_classes"
DST_PATH = "/kaggle/working/cars_75_classes"

os.makedirs(DST_PATH, exist_ok=True)

all_classes = [d for d in os.listdir(SRC_PATH) if d.startswith("class_")]
selected_classes = random.sample(all_classes, 75)

for cls in selected_classes:
    src_folder = os.path.join(SRC_PATH, cls)
    dst_folder = os.path.join(DST_PATH, cls)
    if not os.path.exists(dst_folder):
        shutil.copytree(src_folder, dst_folder)

print("✅ Created dataset with 75 classes at:", DST_PATH)


✅ Created dataset with 75 classes at: /kaggle/working/cars_75_classes


In [ ]:
# ======================================
# 3️⃣ Build DataFrame
# ======================================
DATA_PATH = "/kaggle/working/cars_75_classes"
assert os.path.exists(DATA_PATH), "Dataset not found"

data = []
for cls in sorted(os.listdir(DATA_PATH)):
    if not cls.startswith("class_"):
        continue
    cls_path = os.path.join(DATA_PATH, cls)
    for img in os.listdir(cls_path):
        if img.lower().endswith(('.jpg','.jpeg','.png')):
            data.append({"filename": os.path.join(cls_path, img), "label": cls})

df = pd.DataFrame(data)
print("Total images:", len(df))
print("Total classes:", df["label"].nunique())


Total images: 3175
Total classes: 75


In [ ]:
# ======================================
# 4️⃣ Train / Validation Split
# ======================================
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.8,1.2]
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)
print("NUM_CLASSES:", NUM_CLASSES)


Found 2540 validated image filenames belonging to 75 classes.
Found 635 validated image filenames belonging to 75 classes.
NUM_CLASSES: 75


In [ ]:
# ======================================
# 5️⃣ Build Model (ResNet50)
# ======================================
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(base_model.input, output)

model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", TopKCategoricalAccuracy(k=5, name="top5_accuracy")]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,675,275 (94.13 MB)

 Trainable params: 1,087,563 (4.15 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# ======================================
# 6️⃣ Training Stage 1 (Feature Extraction)
# ======================================
callbacks = [
    EarlyStopping(patience=8, restore_best_weights=True),
    ReduceLROnPlateau(patience=4, factor=0.2, min_lr=1e-6),
    ModelCheckpoint("resnet50_best_stage1.keras", monitor="val_accuracy", save_best_only=True)
]

history_1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    callbacks=callbacks
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 74s 769ms/step - accuracy: 0.0264 - loss: 4.5239 - top5_accuracy: 0.1010 - val_accuracy: 0.0756 - val_loss: 4.0110 - val_top5_accuracy: 0.2661 - learning_rate: 1.0000e-04
Epoch 2/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 48s 605ms/step - accuracy: 0.0738 - loss: 4.0359 - top5_accuracy: 0.2270 - val_accuracy: 0.1465 - val_loss: 3.6990 - val_top5_accuracy: 0.4094 - learning_rate: 1.0000e-04
Epoch 3/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 48s 601ms/step - accuracy: 0.1239 - loss: 3.7423 - top5_accuracy: 0.3450 - val_accuracy: 0.2047 - val_loss: 3.3935 - val_top5_accuracy: 0.4866 - learning_rate: 1.0000e-04
Epoch 4/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 49s 613ms/step - accuracy: 0.1915 - loss: 3.4154 - top5_accuracy: 0.4319 - val_accuracy: 0.2661 - val_loss: 3.1171 - val_top5_accuracy: 0.5748 - learning_rate: 1.0000e-04
Epoch 5/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 48s 599ms/step - accuracy: 0.2255 - loss: 3.2080 - top5_accuracy: 0.4917 - val_accuracy: 0.3071 - val_loss: 2.9242 - val_top5_

In [ ]:
# ======================================
# 7️⃣ Fine-Tuning Stage 2
# ======================================
for layer in base_model.layers[-160:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", TopKCategoricalAccuracy(k=5, name="top5_accuracy")]
)

callbacks_ft = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(patience=5, factor=0.2, min_lr=1e-7),
    ModelCheckpoint("resnet50_best_stage2.keras", monitor="val_accuracy", save_best_only=True)
]

history_2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=callbacks_ft
)


Epoch 1/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.4655 - loss: 2.0079 - top5_accuracy: 0.7794 - val_accuracy: 0.4693 - val_loss: 1.9960 - val_top5_accuracy: 0.7606 - learning_rate: 1.0000e-05
Epoch 2/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 55s 682ms/step - accuracy: 0.6049 - loss: 1.4607 - top5_accuracy: 0.8796 - val_accuracy: 0.5370 - val_loss: 1.6869 - val_top5_accuracy: 0.8205 - learning_rate: 1.0000e-05
Epoch 3/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 58s 724ms/step - accuracy: 0.6791 - loss: 1.2060 - top5_accuracy: 0.9223 - val_accuracy: 0.5606 - val_loss: 1.5381 - val_top5_accuracy: 0.8504 - learning_rate: 1.0000e-05
Epoch 4/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 59s 742ms/step - accuracy: 0.7375 - loss: 1.0348 - top5_accuracy: 0.9443 - val_accuracy: 0.5969 - val_loss: 1.4539 - val_top5_accuracy: 0.8646 - learning_rate: 1.0000e-05
Epoch 5/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 57s 715ms/step - accuracy: 0.7331 - loss: 0.9819 - top5_accuracy: 0.9511 - val_accuracy: 0.6126 - val_loss: 1.3750 - val_top5_ac

In [ ]:
# ======================================
# 8️⃣ Final Evaluation
# ======================================
val_loss, val_acc, val_top5 = model.evaluate(val_gen)
print("Validation Accuracy:", val_acc)
print("Validation Top-5 Accuracy:", val_top5)


20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 184ms/step - accuracy: 0.8349 - loss: 0.6314 - top5_accuracy: 0.9510
Validation Accuracy: 0.8157480359077454
Validation Top-5 Accuracy: 0.9543306827545166
